<a href="https://colab.research.google.com/github/arjasc5231/Lingometer/blob/speaker_verification/speaker_verification/loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
import numpy as np
import tensorflow as tf

In [ ]:
"""
keras에서 custom loss 만드는 법 아래 블로그 참고
https://blog.naver.com/PostView.nhn?blogId=siniphia&logNo=221972407157
"""
def normalize(x):
    return x/tf.sqrt(tf.reduce_sum(x**2, axis=-1, keepdims=True)+1e-6)


def simMat_loss(y_true, y_pred):
  N = 5  # 화자 수
  M = 5  # 화자당 발화 수

  embedded_split = tf.reshape(normalize(y_pred), shape=[N, M, -1])

  center = normalize(tf.reduce_mean(embedded_split, axis=1))  # 화자마다 모든 발화의 평균. [N][P]
  center_except = normalize(tf.reshape(tf.reduce_sum(embedded_split, axis=1, keepdims=True)- embedded_split, shape=[N*M,-1]))  # 각 발화에 대해, 그 발화를 제외한 화자 내의 평균. [N][M][P]
  
  S = tf.concat(
      [tf.concat([tf.reduce_sum(center_except[i*M:(i+1)*M,:]*embedded_split[j,:,:], axis=1, keepdims=True) if i==j
                  else tf.reduce_sum(center[i:(i+1),:]*embedded_split[j,:,:], axis=1, keepdims=True) for i in range(N)],
                  axis=1) for j in range(N)], axis=0)

  S_correct = tf.concat([S[i*M:(i+1)*M, i:(i+1)] for i in range(N)], axis=0)
  total = -tf.reduce_sum(S_correct-tf.math.log(tf.reduce_sum(tf.exp(S), axis=1, keepdims=True) + 1e-6))
  
  """
  loss를 batch별로 확인하고 싶다면 주석을 풀고 아래 코드를 사용하는데,
  print(total.numpy())
  tensor.numpy()로 실시간으로 tensor의 값을 확인하려면 아래 코드를 처음에 추가해줘야 한다.
  tf.config.run_functions_eagerly(True)
  """

  return total
  # cosine sim을 parametric하게 하는건 아직 구현 안함

In [ ]:
def get_loss(loss_name):
    if loss_name=='cross_entropy': return keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    elif loss_name=='simMat_loss': return simMat_loss